In [1]:
# 모듈 설정
from bs4 import BeautifulSoup
from datetime import datetime
import requests
import pandas as pd
import os
import datetime
from urllib import parse

In [7]:
# 코드 엑셀파일 불러오기
code_df = pd.read_csv('code_a.csv', encoding='cp949')
code_df

,회사명,종목코드
0,GS글로벌,1250
1,HSD엔진,82740
2,KG케미칼,1390
3,LG이노텍,11070
4,OCI,10060
5,SK네트웍스,1740
6,SK이노베이션,96770
7,STX,11810
8,WISCOM,24070
9,갤럭시아에스엠,11420


In [3]:
# 저장파일 함수 지정
def news_crawling(new_code): #new_code = 종목코드
    df_crawling = pd.DataFrame()
    
    
    ###
    temp_page_url = 'https://finance.naver.com/item/news_news.nhn?code={}&page={}&sm=title_entity_id.basic&clusterId='.format(new_code, 1) 
    #request
    
    lastPageResp = requests.get(temp_page_url)
    lastPagehtml = lastPageResp.text
    
    #soup초기화
    soup = BeautifulSoup(lastPagehtml)
    
    href = soup.find('td', class_='pgRR').a['href']
    url = parse.urlparse(href)
    
    #Http URL Parameter를 다루려면 parse_qs() 메서드를 사용
    #parse_qs() 는 지정된 쿼리스트링을 해석하여 dict() 로 반환
    url_query = parse.parse_qs(url.query)
    
    pg = url_query['page'][0]
        
    ##400page가 넘는 경우 반복되는 news만 나오므로 400 = pg
    if int(pg) >= int(401):
        pg = int(400)
    else:
        pg = pg
    
    ###      

    for page in range(1,int(pg)+1):
        #종목에서 더 많은 페이지를 설정하는 경우, 마지막페이지만 계속 반복
        page_url = 'https://finance.naver.com/item/news_news.nhn?code={}&page={}&sm=title_entity_id.basic&clusterId='.format(new_code,page)
        resp = requests.get(page_url)
        html = resp.text
        soup = BeautifulSoup(html)
        
        a_tags = soup.select('a.tit')
        title_text=[]    
        link_text=[]
        for a_tag in a_tags:
            title_text.append(a_tag.text)
            link_text.append(a_tag['href'])

        td_tags = soup.select('td.date')   
        date_text=[]             
        for td_tag in td_tags:
            date_text.append(td_tag.string)  #.string하니까 시간만 정제됨!!!

        test_dict={}
        test_dict['date']=date_text
        test_dict['link']=link_text
        test_dict['title']=title_text  #title이라는 키값을 가지는 title_text 리스트의 요소들은  dict로 전환
    
        df_crawling = df_crawling.append(pd.DataFrame(test_dict), ignore_index=True)
    return df_crawling

In [4]:
def report_crawling(new_code):
    
    period_year = 1      #오늘로부터 기간(year) 
    end_day = str(datetime.date.today())[:10]
    start_day = str(int(end_day[:4])-period_year)+end_day[4:]
    domain_url = 'https://www.wisereport.co.kr/wiseReport/reports/company.aspx'
    cookies = {
        'ASP.NET_SessionId': 'm3o23vcypu5mdqb2zz5spjzc'
    }

    page_num=0
    report_list = []
    for i in range(1,10000):
        form_data = {
            'startDT': start_day,
            'endDt': end_day,
            'langTyp': 1,
            'searchTyp': None,
            'searchVal': None,
            'reVal': 'and a.cmp_cd = |{}|'.format(new_code),
            'perPage': 25,
            'curPage': i,
            'sortCol': 'anl_dt',
            'sortTyp': 'd',
            'flag': None,
            'flashYN': 1,
            'userId': 'gaps5_000543'
        }
        resp = requests.post(domain_url, cookies = cookies, data = form_data)
        soup = BeautifulSoup(resp.content,'html.parser')
        tg_contents = soup.find('tbody')

        if soup.find('tbody').find('tr').find('td').attrs['class'] == ['lst_center', 'end']:
            break
        else:
            page_num+=1
#             print('{}페이지를 가져오겠습니다.'.format(page_num))
    
        new_list=[]
        for content in tg_contents.contents[:-2]:
        
            new_dict = {}

            new_dict['title'] = content.find_all('td')[1].find('div').find('a').text

            new_dict['time']  = content.find('td').text

            summary_tag = content.find_all('td')[1].attrs['data-content']
            summary_soup = BeautifulSoup(summary_tag)
            summary_chunk = summary_soup.text
            summary_list = summary_chunk.split('▶')[1:]
            for i in range(len(summary_list)-1):
                summary_list[i] = summary_list[i][:-2]
            new_dict['summary'] = summary_list

            new_list.append(new_dict)

        report_list.extend(new_list)
        
    report_list = pd.DataFrame(report_list)
    return report_list

In [5]:
def price_crawling(new_code):
    price_list = []
    url = 'https://finance.naver.com/item/sise_day.nhn'
    for num in range(1,25):
        params = {
            'code' : new_code,
            'page' : '{}'.format(num)
        }
        resp = requests.get(url, params = params)
        soup = BeautifulSoup(resp.content, 'html.parser')
        for tr in soup.find_all('tr')[2:14]:
            new_dict = {}
            td_list = tr.find_all('td')
            if not td_list[0].text.strip():
                continue
            tdtext_list = ['날짜', '종가', '전일비', '시가', '고가', '저가', '거래량']
            for i in range(7):
                new_dict[tdtext_list[i]] = td_list[i].text.strip()
            price_list.append(new_dict)

    return pd.DataFrame(price_list)

In [6]:
# 회사이름 쳐서 코드 불러오기
name = input('회사이름을 적어주세요 : ')
new_name = name.upper()
a = code_df[code_df['회사명'].isin([new_name])]

while True:
    if a.size == 0:
        print('정확한 회사명을 입력하세요')
        # 특정 문자열이 포함된 리스트 뽑기
        compony_ex = code_df[code_df.회사명.str.contains(new_name) == True]
        print(compony_ex)
        name = input('회사이름을 적어주세요 : ')
        new_name = name.upper()
        a = code_df[code_df['회사명'].isin([new_name])]
    else:
        code = a['종목코드'].values[0]
        new_code = '%06d' %code
        break

    
#뉴스크롤링
news = news_crawling(new_code)

#가격크롤링
price = price_crawling(new_code)

# 리포트크롤링
report = report_crawling(new_code)


#저장, 종료 여부 묻기
while True:
    try:
        num=int(input("저장하시겠습니까?(1-저장, 2-종료) : "))
        print(num)
    
        if not num in [1,2]:
            print("잘못된 명령입니다. 다시 입력해주세요.")
            continue
        
        elif num == 1 :

            BASE_DIR = os.getcwd()
            SAVE_DB_DIR = os.path.join(BASE_DIR, new_name)
    
            if not os.path.exists(SAVE_DB_DIR):
                os.makedirs(SAVE_DB_DIR)
             
            temp_name = f"{new_name}_{new_code}_{str(datetime.date.today())}_report.csv"
            txt_name = os.path.join(SAVE_DB_DIR, temp_name)
            report.to_csv(txt_name, encoding='cp949')
            
            temp_name = f"{new_name}_{new_code}_{str(datetime.date.today())}_price.csv"
            txt_name = os.path.join(SAVE_DB_DIR, temp_name)
            price.to_csv(txt_name, encoding='cp949')
            
            temp_name = f"{new_name}_{new_code}_{str(datetime.date.today())}_news.xlsx"
            txt_name = os.path.join(SAVE_DB_DIR, temp_name)
            news.to_excel(txt_name, sheet_name='sheet1')
        
            print("저장 완료. 프로그램을 종료합니다.")
            break    
    
        else:
            print("프로그램을 종료합니다.")
            break
        
    except ValueError:
        print("잘못된 명령입니다. 다시 입력해주세요.")
        continue

회사이름을 적어주세요 : 삼성
정확한 회사명을 입력하세요
            회사명    종목코드
32        삼성SDI    6400
33     삼성바이오로직스  207940
34        삼성출판사   68290
35         삼성카드   29780
36     삼성화재해상보험     810
626     삼성에스디에스   18260
627       삼성중공업   10140
628        삼성증권   16360
918        삼성물산   28260
919        삼성전자    5930
920        삼성제약    1360
1174       삼성공조    6660
1175       삼성생명   32830
1176       삼성전기    9150
1455    삼성엔지니어링   28050
1564  삼성머스트스팩3호  309930
1856     삼성스팩2호  291230
회사이름을 적어주세요 : 삼성전기
저장하시겠습니까?(1-저장, 2-종료) : 1
1


ModuleNotFoundError: No module named 'openpyxl'

In [11]:
report

,summary,time,title
0,"[ ’18년 실적 기준 동사 전사 매출의 35%가 삼성전자향으로 발생, 다만 ML...",19/06/05 10:13 19/06/05,시간이 더 필요
1,[ 불확실성으로 인한 수요 위축이 주가에 반영. 당분간 불확실성에 인한 악재 반영과...,19/06/05 09:33 19/06/04,남은 트리거는 하나
2,"[ 삼성전기에 대해 6개월 목표주가 145,000원 (2019년 예상 BPS에 P/...",19/06/04 10:31 19/06/04,장기적인 반사이익 예상
3,[ 삼성전기를 당사 커버리지에서 제외],19/06/03 15:07 19/06/03,Coverage 제외
4,"[ 수요 개선 지연, MLCC 가격 하락 등 우려가 큰 상황, 투자의견 `매수` ...",19/06/03 13:26 19/06/03,우려가 반영되고 있는 주가. 우려가 크나 저평가 상태에 진입한 것도 사실
5,"[ 투자의견 매수(BUY) 및 목표주가 150,000원 유지 , 영업실적 및 주요...",19/05/29 19:04 19/05/29,"MLCC, 믿고 가자"
6,[ 하반기 MLCC 수요 회복 전망],19/05/28 10:48 19/05/28,MLCC 5G 사이클 초점
7,"[ 고압 고용량 하이엔드 MLCC는 5G, 전장 등 필수, 카메라 Spec 업그레...",19/05/27 08:14 19/05/27,선택과 집중
8,"[ MLCC 연착륙 기대, 멀티카메라 트렌드 긍정적, 2019년 연간 영업이익 ...",19/05/22 07:08 19/05/22,"MLCC 연착륙, 2020년에 거는 기대"
9,"[ MLCC business likely to have soft landing, ...",19/05/24 11:09 19/05/22,Mechanics]MLCC soft landing; 2020 outlook
